In [6]:
import numpy as np
import torch 
import sys
import matplotlib.pyplot as plt

sys.path.insert(0, 'src/')

from src.model.config import StableDiffusionConfig
from src.model.clip import CLIPEncoder
from src.model.diffusion import StableDiffusion
from src.utils import forward_diffusion, convert_tensor_to_image
from src.model.dataset import get_dataloader
from src.model.tokenizer import Tokenizer
from src.model.dataset import SD_Dataset

from PIL import Image

from torchinfo import summary

from torch.nn.utils.rnn import pad_sequence

from torchvision.transforms.functional import pil_to_tensor, to_pil_image
from torchvision.transforms import Compose, Lambda, Resize, ToPILImage, ToTensor

config = StableDiffusionConfig()

In [7]:
dl = get_dataloader(config)

In [4]:
model = StableDiffusion(config)

In [5]:
sum(p.numel() for p in model.parameters())

924605735